# 17Lands Comparing users and non-users via simulation

__Data source__ - 17Lands individual card ratings (example https://www.17lands.com/card_ratings)<br>
Data is scraped from the website by:
1. Manually visiting the site
2. Setting desired parameters
3. Manually copying the whole table, pasting into excel and saving as an MS-DOS CSV

In [30]:
import pandas as pd
card_ratings = pd.read_csv("AFR_CardRankings_Aug.csv")
card_ratings.dropna(inplace=True)
card_ratings.reset_index(drop=True,inplace=True)
card_ratings.head()

,Name,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD
0,+2 Mace,W,C,69713.0,6.99,8445.0,10.05,14336.0,52.60%,2556.0,50.00%,4121.0,50.00%,6677.0,50.00%,7988.0,54.60%,-4.7pp
1,Arborea Pegasus,W,C,57932.0,5.22,11074.0,6.95,36178.0,54.10%,8012.0,53.10%,13070.0,55.50%,21082.0,54.60%,17990.0,53.60%,1.0pp
2,Blink Dog,W,U,24395.0,6.39,3187.0,9.34,4957.0,51.10%,972.0,47.20%,1261.0,51.80%,2233.0,49.80%,2843.0,52.00%,-2.2pp
3,Celestial Unicorn,W,C,54473.0,5.19,9829.0,6.81,27928.0,54.80%,7391.0,55.70%,11624.0,55.40%,19015.0,55.50%,12646.0,53.60%,1.9pp
4,Cleric Class,W,U,15228.0,4.04,3761.0,4.75,13780.0,56.00%,2680.0,55.00%,4726.0,58.40%,7406.0,57.20%,6898.0,55.10%,2.1pp


### Cleaning fields of interest

Percentage fields all need to drop the % sign, and optionally divide by 100<br>
IWD needs to drop the "pp" text<br>
All the "#" fields are integers can drop the decimal and be recast

In [31]:
#card_ratings.dtypes
card_ratings["# Seen"] = card_ratings["# Seen"].astype(int)
card_ratings["# Picked"] = card_ratings["# Picked"].astype(int)
card_ratings["# GP"] = card_ratings["# GP"].astype(int)
card_ratings["GP WR"] = card_ratings["GP WR"].str.replace("%","").astype(float) / 100
card_ratings["# OH"] = card_ratings["# OH"].astype(int)
card_ratings["OH WR"] = card_ratings["OH WR"].str.replace("%","").astype(float) / 100
card_ratings["# GD"] = card_ratings["# GD"].astype(int)
card_ratings["GD WR"] = card_ratings["GD WR"].str.replace("%","").astype(float) / 100
card_ratings["# GIH"] = card_ratings["# GIH"].astype(int)
card_ratings["GIH WR"] = card_ratings["GIH WR"].str.replace("%","").astype(float) / 100
card_ratings["# GND"] = card_ratings["# GND"].astype(int)
card_ratings["GND WR"] = card_ratings["GND WR"].str.replace("%","").astype(float) / 100
card_ratings["IWD"] = card_ratings["IWD"].str.replace("pp","").astype(float)

#card_ratings.dtypes
card_ratings.head()

,Name,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD
0,+2 Mace,W,C,69713,6.99,8445,10.05,14336,0.526,2556,0.500,4121,0.500,6677,0.500,7988,0.546,-4.7
1,Arborea Pegasus,W,C,57932,5.22,11074,6.95,36178,0.541,8012,0.531,13070,0.555,21082,0.546,17990,0.536,1.0
2,Blink Dog,W,U,24395,6.39,3187,9.34,4957,0.511,972,0.472,1261,0.518,2233,0.498,2843,0.520,-2.2
3,Celestial Unicorn,W,C,54473,5.19,9829,6.81,27928,0.548,7391,0.557,11624,0.554,19015,0.555,12646,0.536,1.9
4,Cleric Class,W,U,15228,4.04,3761,4.75,13780,0.560,2680,0.550,4726,0.584,7406,0.572,6898,0.551,2.1


## Super-simple comparison - what are the top commons?

In [103]:
df = card_ratings.copy() # for ease of coding and prevents us from accidentally polluting the original

# Gameplay ranking can be estimated via GIH WR (game in hand winrate)
# 17Lands users ranking can be seen via ATA (average taken at)
# Non-17Lands users ranking can be seen via ALSA (average last seen at)

df = df[df.Color.isin(["W","U","B","R","G"])]

gih = df[df.Rarity == "C"].groupby(["Color"])["Name","Color","GIH WR"].apply(lambda x: x.nlargest(5, columns="GIH WR"))
ata = df[df.Rarity == "C"].groupby(["Color"])["Name","Color","ATA"].apply(lambda x: x.nsmallest(5, columns="ATA"))
alsa = df[df.Rarity == "C"].groupby(["Color"])["Name","Color","ALSA"].apply(lambda x: x.nsmallest(5, columns="ALSA"))
commons = pd.DataFrame()
commons["Color"] = gih["Color"].values
commons["GIH"] = gih["Name"].values
commons["17LandsUsers"] = ata["Name"].values
commons["Non17LandsUsers"] = alsa["Name"].values

gih = df[df.Rarity == "U"].groupby(["Color"])["Name","Color","GIH WR"].apply(lambda x: x.nlargest(5, columns="GIH WR"))
ata = df[df.Rarity == "U"].groupby(["Color"])["Name","Color","ATA"].apply(lambda x: x.nsmallest(5, columns="ATA"))
alsa = df[df.Rarity == "U"].groupby(["Color"])["Name","Color","ALSA"].apply(lambda x: x.nsmallest(5, columns="ALSA"))
uncommons = pd.DataFrame()
uncommons["Color"] = gih["Color"].values
uncommons["GIH"] = gih["Name"].values
uncommons["17LandsUsers"] = ata["Name"].values
uncommons["Non17LandsUsers"] = alsa["Name"].values

In [104]:
commons

,Color,GIH,17LandsUsers,Non17LandsUsers
0,B,Grim Bounty,Grim Bounty,Grim Bounty
1,B,Vampire Spawn,Precipitous Drop,Precipitous Drop
2,B,Deadly Dispute,Yuan-Ti Fang-Blade,Yuan-Ti Fang-Blade
3,B,Shambling Ghast,Shambling Ghast,Shambling Ghast
4,B,Precipitous Drop,Sepulcher Ghoul,Hoard Robber
5,G,Owlbear,Owlbear,Owlbear
6,G,Hill Giant Herdgorger,Spoils of the Hunt,Spoils of the Hunt
7,G,Spoils of the Hunt,Gnoll Hunter,Gnoll Hunter
8,G,Gnoll Hunter,Underdark Basilisk,Underdark Basilisk
9,G,Elturgard Ranger,Hill Giant Herdgorger,Hill Giant Herdgorger


## And uncommons

In [105]:
uncommons

,Color,GIH,17LandsUsers,Non17LandsUsers
0,B,Skullport Merchant,Power Word Kill,Power Word Kill
1,B,Power Word Kill,Reaper's Talisman,Reaper's Talisman
2,B,Warlock Class,Skullport Merchant,Skullport Merchant
3,B,Black Dragon,Black Dragon,Black Dragon
4,B,Reaper's Talisman,Death-Priest of Myrkul,Death-Priest of Myrkul
5,G,Hunter's Mark,Prosperous Innkeeper,Prosperous Innkeeper
6,G,Prosperous Innkeeper,Wandering Troubadour,Wandering Troubadour
7,G,Intrepid Outlander,Hunter's Mark,Hunter's Mark
8,G,Lurking Roper,Intrepid Outlander,Intrepid Outlander
9,G,Wandering Troubadour,Purple Worm,Purple Worm
